In [2]:
# Packages for analysis
import pandas as pd
import numpy as np
import pyart
import numpy.ma as ma
import os
import warnings
warnings.filterwarnings('ignore')

In [6]:
class UF:
    def __init__(self,radar,shape_grid,lat_0, lon_0):
        self.radar = radar
        self.shape_grid = shape_grid
        self.lat_0 = lat_0
        self.lon_0 = lon_0
    def remove_noises(self): #Step 3-A
        mask_noises_CC=ma.masked_less(self.radar.fields['cross_correlation_ratio']['data'],0.85)
        mask_noises_Z=ma.masked_less(self.radar.fields['reflectivity']['data'],0)
        rm_noises_Z_=np.ma.masked_array(self.radar.fields['reflectivity']['data'],mask_noises_CC.mask)
        rm_noises_Z=np.ma.masked_array(rm_noises_Z_,mask_noises_Z.mask)
        rm_noises_ZDR_=np.ma.masked_array(self.radar.fields['differential_reflectivity']['data'],mask_noises_CC.mask)
        rm_noises_ZDR=ma.masked_array(rm_noises_ZDR_,mask_noises_Z.mask)
        self.radar.add_field_like('reflectivity','Z_removed_noises',rm_noises_Z,replace_existing=True)
        self.radar.add_field_like('differential_reflectivity','ZDR_removed_noises', rm_noises_ZDR,replace_existing=True)
    def convert_grid(self):
        grid = pyart.map.grid_from_radars(
            self.radar,
            grid_shape=self.shape_grid, #Number of points in the grid (z, y, x)
            grid_limits=((0, 9000), (-200000, 200000), (-200000, 200000)), # min-max tuong duong z,y,x
            grid_origin = (self.lat_0, self.lon_0),
            fields=['Z_removed_noises',
                    'ZDR_removed_noises'],
            roi_func='dist_beam',
            weighting_function='cressman')
        return grid
    def convert_lat_lon(self, grid):
        #conver distance to lat/lon
        for i in range(self.shape_grid[1]):
            geog = pyart.core.cartesian_to_geographic_aeqd(grid.x["data"][i],grid.y["data"][i],self.lon_0, self.lat_0, R=6370997.0)
            grid.x["data"][i] = geog[0]
            grid.y["data"][i] = geog[1]
        return grid
    def YH95_algorithm(self,Zh):
        Zh.mask=0
        YH95=np.zeros((self.shape_grid[1],self.shape_grid[2]))
        #Step 1: Intensity
        mask_st1=(Zh>=40)
        YH95[mask_st1]=1
        #Step 2: Peakeness
        def MBG(xo,yo,n,r,Zh): #Mean Background Reflectivity
            y,x = np.ogrid[-xo:n-xo, -yo:n-yo]
            mask = x*x + y*y <= r*r
            Zh_none_O=Zh[xo,yo]
            Zbg_ = np.ma.masked_array(Zh[mask], Zh[mask] ==Zh_none_O) # remove value cycle centers
            Zbg = np.ma.masked_array(Zbg_, Zbg_ == 0) # remove 0 values
            return Zbg
        for xo in range (0,self.shape_grid[1],1):
            for yo in range (0,self.shape_grid[2],1):
                if YH95[xo,yo]==0:
                    Zbg=MBG(xo,yo,self.shape_grid[1],5.5,Zh).mean()
                    deltaZh=Zh[xo,yo]-Zbg
                    if (Zbg < 42.43) and (deltaZh >= (10-Zbg**2/180)):
                        YH95[xo,yo]=1
                    elif Zbg>=42.43 and deltaZh>=0:
                        YH95[xo,yo]=1
        #Step3: Surrounding area
        def MBG_mask_r(xo,yo,n,r,): # mask array with r change
            y,x = np.ogrid[-xo:n-xo, -yo:n-yo]
            mask = x*x + y*y <= r*r
            return mask
        medium=[25,30,35,40]
        for xo in range (0,self.shape_grid[1],1):
            for yo in range (0,self.shape_grid[2],1):
                if YH95[xo,yo]==1:
                    Zbg=MBG(xo,yo,self.shape_grid[1],5.5,Zh).mean()
                    if Zbg < medium[0]:
                        r=0.5 #1km
                    elif Zbg>=medium[0] and Zbg <medium[1]:
                        r=1   #2km
                    elif Zbg>=medium[1] and Zbg <medium[2]: 
                        r=1.5 #3km
                    elif Zbg>=medium[2] and Zbg <medium[3]:
                        r=2   #4km
                    elif Zbg>=medium[3]:
                        r=2.5 #5km
                    mask_st2=MBG_mask_r(xo,yo,self.shape_grid[1],r=r)
                    YH95[mask_st2]=2
         #Step4: remaining areas as stratiform
        for q in range (0,self.shape_grid[1],1):
            for l in range (0,self.shape_grid[2],1):
                if Zh[q,l] > 0 and YH95[q,l] !=2:
                    YH95[q,l]=1
        return YH95
#Tạo giới hạn bán kính 200km    
y,x = np.ogrid[-100:201-100, -100:201-100]
mask = x*x + y*y > 100*100

In [8]:
df=pd.DataFrame(data={'Type':[],'Z':[],'ZDR':[]})
Cases=['Squall line','Full stratiform','Embedded',]
for Case in Cases:
    print(Case)
    linkRAW='C:/Users/Admin/OneDrive/Desktop/Practice SVM/Frequency/A2/6 events datas(2)/'+Case+'/'
    for file_name in os.listdir(linkRAW):
        radar = pyart.io.read_sigmet(linkRAW+file_name)
        lat_0 = radar.latitude['data'][0]
        lon_0 = radar.longitude['data'][0]
        shape_grid = (7, 201,201)
        UF_File = UF(radar, shape_grid, lat_0, lon_0)
        UF_File.remove_noises()
        grid = UF_File.convert_grid()
        grid_lat_lon = UF_File.convert_lat_lon(grid)
        YH95=np.ma.masked_array(ma.masked_equal(UF_File.YH95_algorithm(grid.fields['Z_removed_noises']['data'][1,:,:]),0.0),mask)
        Zh_SVM=np.ma.masked_array(ma.masked_equal(grid.fields['Z_removed_noises']['data'][1,:,:],0.0),mask)
        ZDR_SVM=np.ma.masked_array(grid.fields['ZDR_removed_noises']['data'][1,:,:],mask)
        n=0
        for i in range(0,shape_grid[1],1):
            for j in range(0,shape_grid[2],1):
                if YH95[i,j]==1 and Zh_SVM[i,j]<35 and ZDR_SVM[i,j]<2:
                    n=n+1
                    S=pd.Series(data=['Stratiform',Zh_SVM[i,j],ZDR_SVM[i,j]],index=df.columns,name=n)
                    df=df.append(S)
                if YH95[i,j]==2 and Zh_SVM[i,j]>25 and ZDR_SVM[i,j]>0:
                    n=n+1
                    C=pd.Series(data=['Convection',Zh_SVM[i,j],ZDR_SVM[i,j]],index=df.columns,name=n)
                    df=df.append(C)
        print(file_name)

Squall line
PHA210207132004.RAWSB79
PHA210512140004.RAWX92W
PHA210512194004.RAWX9C5
PHA210519084004.RAW5JGW
PHA210731184003.RAW50FS
PHA220205141004.RAW3SSM
Full stratiform
PHA210208202004.RAWMLM9
PHA210404050004.RAWWUVL
PHA210404112004.RAWWV51
PHA210718050004.RAWKJRX
PHA210802234004.RAWKLCY
PHA210817022003.RAWL467
Embedded
PHA210428144004.RAWN3XN
PHA210505054004.RAWBZW1
PHA210523170004.RAW108Y
PHA210707180003.RAWK6A6
PHA210718012004.RAWKJJX
PHA210724182004.RAWDRMN


In [9]:
df

,Type,Z,ZDR
1,Stratiform,9.850332,0.714538
2,Stratiform,11.518294,1.503283
3,Stratiform,9.977217,0.774539
4,Stratiform,10.679627,1.106694
5,Stratiform,8.734643,1.410818
...,...,...,...
10474,Stratiform,3.504172,-1.101138
10475,Stratiform,2.832242,-1.700585
10476,Stratiform,8.200000,0.230000
10477,Stratiform,6.304914,1.034104


In [10]:
df.to_excel("C:/Users/Admin/OneDrive/Desktop/Practice SVM/Frequency/A2/b4.xlsx")